In [ ]:
import numpy as np
import time
from IPython.display import clear_output
import unittest
import import_ipynb
from mcts import Mcts
from node import Node
from template import Template

In [ ]:
# M# 2 - True (you)
# 1 - False (bot)

class Env(Template):
  """ 
  AI playing TicTacToe as your opponent 
  Solving it using MINMAX Algorithm
  state 1 = X
  state 2 = O
  state 0 = empty block

  """
  def __init__(self):
    self.turn = True
    self.size = 3
    self.state = np.zeros( (self.size,self.size),dtype = np.int8)
    self.bot = 1
    self.count = 0
    self.pos = (2,2)
    # print("ini")


  def isComplete(self,state):
    """ Check wheather game is over and return the winner
    0 if its Draw
    -1 not complete """

    li = [1,2]
    for i in li:
      # horizontal check
      for k in range(self.size):
        res = [True if h == i else False for h in state[k,:] ]
        if all(res):
          return i
      # vertical check
      for k in range(self.size):
        res = [True if h == i else False for h in state[:,k] ]
        if all(res):
          return i

      for k in range(self.size):
        res = [True if state[k][k] == i else False for k in range(self.size) ]
        res1 = [True if state[k][self.size-k-1] == i else False for k in range(self.size) ]
        if all(res) or all(res1):
          return i
    return -1

  def getDragMoves(self, state, player):
    up = [1, -1, 0] 
    down = [1, -1, 0]
    moves = []
    pid = 1 if player else 2
    for i in range(self.size):
      for j in range(self.size):
        if state[i][j] == pid:
#           moves[(i,j)] = []
          for k in up:
            for l in down:
             if k==0 and l==0:
               continue
             if i+k>=0 and j+l>=0 and i+k<self.size and j+l<self.size and state[i+k][j+l]==0:
               moves.append([i,j,i+k, j+l])  
    return moves
        
  
  def getPlaceMove(self, state):
    move = []
    for i in range(self.size):
      for j in range(self.size):
        if(state[i][j] == 0): 
          move.append([-1,-1, i ,j])
    return move

  def getMoves(self, state,turn):
    """
    Return possible moves from the given state
    """
    if self.count>5:
        moves = self.getDragMoves(self, state, turn)
    else:
        moves = self.getPlaceMove(self, state)
    
    return moves
    

  def executeMove(self,state,move,turn):
    #   print("execute move ",turn)
      assert len(state) == 3
      player = 2 if turn else 1
      if move[0] = -1:
        state[move[2]][move[3]] = player
      else:
        state[move[2]][move[3]] = player
        state[move[0]][move[1]] = 0
      return state

  

  def engine(self,debug = False):

      print("turn = ",self.turn)

      n = Node(self,turn= self.turn)
      n.state = self.state.copy()
      n.visited = 1
      n.isRoot = True # can be remobed
      n.isLeafNode = True


      mcts = Mcts(self,self.turn,n,debug = False)

      move =  mcts.execute(True)

      # del mcts
      assert move != None
      if debug:
          return mcts
      return move

  def printState(self,state):

    """ Display the board state node.state"""

    for i in range(self.size-1,-1,-1): # printing states in reverse
      for j in range(self.size): 
        if state[i][j] == 0 :
          curr = ' '
        elif state[i][j] == 1:
          curr = 'X'
        else:
          curr = 'O'  
        if j != self.size-1:
          print(f"{curr} | ",end = '')
        else:
          print(f"{curr}")
      if i != 0:
        print("--"*self.size*2)
                                                                                                                                     

  def play(self,youFirst = True):
    print("\t TicTacToe \n Bot = X \n You = O ")
    self.turn = youFirst
    while True:
      clear_output(True)
      res = self.isComplete(self.state)
      if res in [0,1,2]:
        self.printState(self.state)
        if res == 0:
          print("Draw")
          break
        print(f"Player {res} Won")
        break

      print(f"Player {self.turn} thinking ... ")
      
      self.printState(self.state)
      if self.turn:
          move = self.engine()
#           a = int(input("intput number"))-1
#           x,y = a//3, a%3
#           self.state = self.executeMove(self.state,[x,y],self.turn)
          self.state = self.executeMove(self.state,move,self.turn)
          
      else:
          move = self.engine()
          self.state = self.executeMove(self.state,move,self.turn)

      
      self.turn = not self.turn
    #   time.sleep(0.5)
      
